In [4]:
import json
#from google.colab import files # solo para google collab
from google.cloud import storage
import pandas as pd
print(storage.__version__)

2.6.0


In [5]:
# google collab
#path_api_key = '/content/extreme-unison-399121-cadd77c555ca.json'

# entorno local
path_api_key = '/Users/manuel/gdrive/prgrmmng/gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

In [6]:
# carga de datos
# unicamente para .py en VM:
#data_gmaps_reviews_norm = pd.read_parquet('gs://1_transform/data_gmaps_reviews_norm.parquet')

# para entorno Google Colab o maquina local:

# cambiar segun bucket
bucket = client_storage.get_bucket('1_transform')

# cambiar segun dataframe
data_blob = bucket.blob('data_gmaps_reviews_norm.parquet')
data_gmaps_reviews = pd.read_parquet(storage.fileio.BlobReader(data_blob))
data_gmaps_reviews.info()
data_gmaps_reviews.head(2)

data_gmaps_reviews[["rating"]]

In [ ]:
data_gmaps_reviews["rating"] = data_gmaps_reviews["rating"].groupby(by=["b"]).sum().copy()